### Imports

In [ ]:
import pandas as pd
import glob
import datetime
import os, shutil
import numpy as np
import swifter
import math
import json
import time
from dask import dataframe as dd
from psutil import virtual_memory
import pprint

In [ ]:
def mem():
    print(f'used memory : {round(virtual_memory()[3]/(1024*1024*1024)*10)/10}Go')

In [ ]:
def stats(): 
    print("--- %s seconds ---" % (time.time() - start_time))
    mem()

### Path set-up

In [ ]:
if "DATA_DIR" not in locals():
    DATA_DIR = "./data/"
else:
    print(DATA_DIR)

if os.path.exists(DATA_DIR) and os.path.isdir(DATA_DIR):
    shutil.rmtree(DATA_DIR)
os.makedirs(os.path.dirname(DATA_DIR), exist_ok=True)

In [ ]:
if "OUTPUT_DATA_FOLDER" not in locals():
    OUTPUT_DATA_FOLDER = "./output/"
else:
    print(OUTPUT_DATA_FOLDER)

if os.path.exists(OUTPUT_DATA_FOLDER) and os.path.isdir(OUTPUT_DATA_FOLDER):
    shutil.rmtree(OUTPUT_DATA_FOLDER)
os.makedirs(os.path.dirname(OUTPUT_DATA_FOLDER), exist_ok=True)

In [ ]:
if "LABELS_FOLDER" not in locals():
    LABELS_FOLDER = "./labels/"
else:
    print(LABELS_FOLDER)

In [ ]:
if "ELASTIC_INDEX" not in locals():
    ELASTIC_INDEX = "siren"
else:
    print(ELASTIC_INDEX)

# Établissement

In [ ]:
# Create list of departement zip codes
all_deps = [
    *"-0".join(list(str(x) for x in range(0, 10))).split("-")[1:],
    *list(str(x) for x in range(10, 20)),
    *["2A", "2B"],
    *list(str(x) for x in range(21, 96)),
    *"-7510".join(list(str(x) for x in range(0, 10))).split("-")[1:],
    *"-751".join(list(str(x) for x in range(10, 21))).split("-")[1:],
    *["971", "972", "973", "974", "976"],
    *[""],
]
# Remove Paris zip code
all_deps.remove("75")

In [ ]:
# all_deps = ["23"]

In [ ]:
%%time
# Upload geo data by departement

# we can reduce this to only the download time by scrapping pandas entirey and only saving csv on disk
# I didnot do it as it requires to rewrite column names
# should save ~ 4~5min

for dep in all_deps:
    start_time = time.time()
    url = "https://files.data.gouv.fr/geo-sirene/last/dep/geo_siret_" + dep + ".csv.gz"
    print(url)
    df_dep = pd.read_csv(
        url,
        compression="gzip",
        dtype=str,
        usecols=[
            "siren",
            "siret",
            "dateCreationEtablissement",
            "trancheEffectifsEtablissement",
            "activitePrincipaleRegistreMetiersEtablissement",
            "etablissementSiege",
            "numeroVoieEtablissement",
            "libelleVoieEtablissement",
            "codePostalEtablissement",
            "libelleCommuneEtablissement",
            "libelleCedexEtablissement",
            "typeVoieEtablissement",
            "codeCommuneEtablissement",
            "codeCedexEtablissement",
            "complementAdresseEtablissement",
            "distributionSpecialeEtablissement",
            "complementAdresse2Etablissement",
            "indiceRepetition2Etablissement",
            "libelleCedex2Etablissement",
            "codeCedex2Etablissement",
            "numeroVoie2Etablissement",
            "typeVoie2Etablissement",
            "libelleVoie2Etablissement",
            "codeCommune2Etablissement",
            "libelleCommune2Etablissement",
            "distributionSpeciale2Etablissement",
            "dateDebut",
            "etatAdministratifEtablissement",
            "enseigne1Etablissement",
            "enseigne1Etablissement",
            "enseigne2Etablissement",
            "enseigne3Etablissement",
            "denominationUsuelleEtablissement",
            "activitePrincipaleEtablissement",
            "geo_adresse",
            "geo_id",
            "longitude",
            "latitude",
            "indiceRepetitionEtablissement",
            "libelleCommuneEtrangerEtablissement",
            "codePaysEtrangerEtablissement",
            "libellePaysEtrangerEtablissement",
            "libelleCommuneEtranger2Etablissement",
            "codePaysEtranger2Etablissement",
            "libellePaysEtranger2Etablissement",
        ],
    )
    df_dep = df_dep.rename(
        columns={
            "dateCreationEtablissement": "date_creation",
            "trancheEffectifsEtablissement": "tranche_effectif_salarie",
            "activitePrincipaleRegistreMetiersEtablissement": "activite_principale_registre_metier",
            "etablissementSiege": "is_siege",
            "numeroVoieEtablissement": "numero_voie",
            "typeVoieEtablissement": "type_voie",
            "libelleVoieEtablissement": "libelle_voie",
            "codePostalEtablissement": "code_postal",
            "libelleCedexEtablissement": "libelle_cedex",
            "libelleCommuneEtablissement": "libelle_commune",
            "codeCommuneEtablissement": "commune",
            "complementAdresseEtablissement": "complement_adresse",
            "complementAdresse2Etablissement": "complement_adresse_2",
            "numeroVoie2Etablissement": "numero_voie_2",
            "indiceRepetition2Etablissement": "indice_repetition_2",
            "typeVoie2Etablissement": "type_voie_2",
            "libelleVoie2Etablissement": "libelle_voie_2",
            "codeCommune2Etablissement": "commune_2",
            "libelleCommune2Etablissement": "libelle_commune_2",
            "codeCedex2Etablissement": "cedex_2",
            "libelleCedex2Etablissement": "libelle_cedex_2",
            "codeCedexEtablissement": "cedex",
            "dateDebut": "date_debut_activite",
            "distributionSpecialeEtablissement": "distribution_speciale",
            "distributionSpeciale2Etablissement": "distribution_speciale_2",
            "etatAdministratifEtablissement": "etat_administratif_etablissement",
            "enseigne1Etablissement": "enseigne_1",
            "enseigne2Etablissement": "enseigne_2",
            "enseigne3Etablissement": "enseigne_3",
            "activitePrincipaleEtablissement": "activite_principale",
            "indiceRepetitionEtablissement": "indice_repetition",
            "denominationUsuelleEtablissement": "nom_commercial",
            "libelleCommuneEtrangerEtablissement": "libelle_commune_etranger",
            "codePaysEtrangerEtablissement": "code_pays_etranger",
            "libellePaysEtrangerEtablissement": "libelle_pays_etranger",
            "libelleCommuneEtranger2Etablissement": "libelle_commune_etranger_2",
            "codePaysEtranger2Etablissement": "code_pays_etranger_2",
            "libellePaysEtranger2Etablissement": "libelle_pays_etranger_2",
        }
    )
    df_dep.to_csv(DATA_DIR + "geo_siret_" + dep + ".csv", index=False)
    stats()

In [ ]:
# Get geo data file paths
geo_files = glob.glob(DATA_DIR + "geo_siret*.csv")

In [ ]:
# Does not take into consideration adress 2
def adresse_complete(cols, row, adresse_2=False):
    def get(x, default=None): 
        val = row[cols[x]]
        if not val:
            return default
        return val
    
    col_list = ["complement_adresse", "numero_voie", "indice_repetition", "type_voie", "libelle_voie", "distribution_speciale"]
    adresse = ""
    for column in col_list:
        col_label = f"{column}_2" if adresse_2 else column
        
        adresse = adresse + (" " + str(get(col_label)) if get(col_label) else "")
    if not adresse_2:
        if get("cedex") is None:
            if get("commune") is None:
                adresse =  adresse
            else:
                adresse = adresse + " " + str(get("commune")) + " " + str(get("libelle_commune"))
        else:
            adresse = adresse + " " + str(get("cedex")) + " " + str(get("libelle_cedex"))
        etranger_list = ["libelle_commune_etranger", "libelle_pays_etranger"]
        for column in etranger_list:
            adresse = adresse + (" " + str(get(column)) if get(column) else "")
        return adresse.strip()
    else:
        if get("cedex_2") is None:
            if get("commune_2") is None:
                adresse =  adresse
            else:
                adresse = adresse + " " + str(get("commune_2")) + " " + str(get("libelle_commune_2"))
        else:
            adresse = adresse + " " + str(get("cedex_2")) + " " + str(get("libelle_cedex_2"))
        etranger_list = ["libelle_commune_etranger_2", "libelle_pays_etranger_2"]
        for column in etranger_list:
            adresse = adresse + (" " + str(get(column)) if get(column) else "")
        return adresse.strip()

In [ ]:
def get_key(k, dictio, default=0):
    if(k in dictio):
        return dictio[k]
    return default

In [ ]:
def parse_etab(cols, row, all_unite_legale, index):
    def get(x, default=None): 
        val = row[cols[x]]
        if not val:
            return default
        return val

    siren = get('siren')

    unite_legale = get_key(siren, all_unite_legale, {})

    etablissement = {}
    siret = get("siret")
    etablissement["enseigne"] = " ".join([
                        get("enseigne_1", ""),
                        get("enseigne_2", ""),
                        get("enseigne_3", ""),
                        get("nom_commercial", ""),
                    ]).strip() or None
    etablissement["adresse_complete"] = adresse_complete(cols, row)
    etablissement["adresse_complete_2"] = adresse_complete(cols, row, adresse_2=True)
    # etablissement["adresse"] = get("geo_adresse") # do we use this or adresse_complete ??
    col_list = ["siret", "is_siege", "etat_administratif_etablissement", "date_creation", "tranche_effectif_salarie", "date_debut_activite", "activite_principale" ,
                 "numero_voie", "type_voie", "libelle_voie", "distribution_speciale", "cedex", "libelle_cedex", "commune", "libelle_commune", "code_pays_etranger",
                 "code_postal", "geo_id", "longitude", "latitude", "activite_principale_registre_metier"]
    for col in col_list:
        etablissement[col] = get(col)
    etablissement["departement"] = str(etablissement["commune"])[:3] if str(etablissement["commune"])[:2]== "97" else (None if etablissement["commune"] is None else str(etablissement["commune"])[:2])
    etablissement["coordonnees"] = None if (etablissement["longitude"] is None) or (etablissement["latitude"] is None) else f'{etablissement["latitude"]},{etablissement["longitude"]}'
    
    
    # etablissements only has one etab
    if "etablissements" not in unite_legale:
        unite_legale["etablissements"]= {}
    unite_legale["etablissements"][siret] = etablissement
    unite_legale["nombre_etablissements"] = get_key("nombre_etablissements", unite_legale, 0) + 1
    
    # adresse complete au niveau de l'établissement et pas l'unité légale
    '''
    if is_siege:
        etablissement["adresse_complete"] = adresse_complete(cols, row)
        etablissement["adresse_complete_2"] = adresse_complete(cols, row, adresse_2=True)
    '''
    is_etablissement_ouvert = etablissement["etat_administratif_etablissement"] == "A"
    
    
    if is_etablissement_ouvert: 
        unite_legale["nombre_etablissements_ouverts"] = get_key("nombre_etablissements_ouverts", unite_legale, 0) + 1
    else:
        unite_legale["nombre_etablissements_ouverts"] = get_key("nombre_etablissements_ouverts", unite_legale, 0)
        
    
    all_unite_legale[siren] = unite_legale
    
#    if(index % 10000 == 0):
#        print(unite_legale)

In [ ]:
%%time
from csv import reader

all_unite_legale = {}

for geo_file in geo_files:
    cols = {}
    with open(geo_file, 'r') as read_obj:
        # pass the file object to reader() to get the reader object
        csv_reader = reader(read_obj)
        # Iterate over each row in the csv using reader object
        for index, row in enumerate(csv_reader):
            if(index==0):
                cols = { val: index for (index, val) in enumerate(row)}
            else:
                parse_etab(cols, row, all_unite_legale, index)
            if(index % 1000000 == 0):
                mem()

In [ ]:
for unite_legale in all_unite_legale.values():
    unite_legale["liste_enseigne"] = list(filter(None, set(etab["enseigne"] for etab in unite_legale["etablissements"].values())))
    unite_legale["liste_adresse"] = list(filter(None, set(etab["adresse_complete"] for etab in unite_legale["etablissements"].values())))
    unite_legale["concat_enseigne_adresse"] = unite_legale["liste_enseigne"] + unite_legale["liste_adresse"]

In [ ]:
pprint.pprint(list(all_unite_legale.values())[0:5])

In [ ]:
len(all_unite_legale)

In [ ]:
for siren,unite_legale in all_unite_legale.copy().items():
    for siret, etablissement in unite_legale["etablissements"].copy().items():
        if etablissement["is_siege"] == "false":
            del unite_legale["etablissements"][siret]

In [ ]:
pprint.pprint(list(all_unite_legale.values())[0:5])

# Unité Légale

In [ ]:
from os.path import exists

unite_file = DATA_DIR + "unite_legales.csv"

unite_legale_file_exists = exists(unite_file)

In [ ]:
%%time

# Import Stock Unite Legale data

# same a etablissement we can reduce this to only the download time by scrapping pandas entirey and only saving csv on disk
# I didnot do it as it requires to rewrite column names
# should save ~1-2min

if not unite_legale_file_exists:
    start_time = time.time()
    chunk = pd.read_csv(
        "https://files.data.gouv.fr/insee-sirene/StockUniteLegale_utf8.zip",
        compression="zip",
        dtype=str,
        chunksize=100000,
        usecols=[
            "siren",
            "dateCreationUniteLegale",
            "sigleUniteLegale",
            "prenom1UniteLegale",
            "identifiantAssociationUniteLegale",
            "trancheEffectifsUniteLegale",
            "dateDernierTraitementUniteLegale",
            "categorieEntreprise",
            "etatAdministratifUniteLegale",
            "nomUniteLegale",
            "nomUsageUniteLegale",
            "denominationUniteLegale",
            "categorieJuridiqueUniteLegale",
            "activitePrincipaleUniteLegale",
            "economieSocialeSolidaireUniteLegale",
        ],
    )
    # Rename columns
    df_unite_legale = pd.concat(chunk)
    df_unite_legale = df_unite_legale.rename(
        columns={
            "dateCreationUniteLegale": "date_creation_unite_legale",
            "sigleUniteLegale": "sigle",
            "prenom1UniteLegale": "prenom",
            "trancheEffectifsUniteLegale": "tranche_effectif_salarie_unite_legale",
            "dateDernierTraitementUniteLegale": "date_mise_a_jour_unite_legale",
            "categorieEntreprise": "categorie_entreprise",
            "etatAdministratifUniteLegale":"etat_administratif_unite_legale",
            "nomUniteLegale": "nom",
            "nomUsageUniteLegale": "nom_usage",
            "denominationUniteLegale": "nom_raison_sociale",
            "categorieJuridiqueUniteLegale": "nature_juridique_unite_legale",
            "activitePrincipaleUniteLegale": "activite_principale_unite_legale",
            "economieSocialeSolidaireUniteLegale":"economie_sociale_solidaire_unite_legale",
            "identifiantAssociationUniteLegale":"identifiant_association_unite_legale",
        }
    )
    df_unite_legale.to_csv(unite_file, index=False)
    stats()

In [ ]:
df_unite_legale.head()


### Data preprocessing

In [ ]:
sections_NAF = {
"01":"A","02":"A","03":"A","05":"B","06":"B","07":"B","08":"B","09":"B","10":"C","11":"C","12":"C","13":"C","14":"C",
 "15":"C","16":"C","17":"C","18":"C","19":"C","20":"C","21":"C","22":"C","23":"C","24":"C","25":"C","26":"C","27":"C",
 "28":"C","29":"C","30":"C","31":"C","32":"C","33":"C","35":"D","36":"E","37":"E","38":"E","39":"E","41":"F","42":"F",
 "43":"F","45":"G","46":"G","47":"G","49":"H","50":"H","51":"H","52":"H","53":"H","55":"I","56":"I","58":"J","59":"J",
 "60":"J","61":"J","62":"J","63":"J","64":"K","65":"K","66":"K","68":"L","69":"M","70":"M","71":"M","72":"M","73":"M",
 "74":"M","75":"M","77":"N","78":"N","79":"N","80":"N","81":"N","82":"N","84":"O","85":"P","86":"Q","87":"Q","88":"Q",
 "90":"R","91":"R","92":"R","93":"R","94":"S","95":"S","96":"S","97":"T","98":"T","99":"U"
}

In [ ]:
def nom_complet(cols, row):
    def get(x, default=None): 
        val = row[cols[x]]
        if not val:
            return default
        return val
    
    is_auto_entrepreneur = get("nature_juridique_unite_legale") == "1000"
    
    sigle = get("sigle")
    
    if is_auto_entrepreneur:
        prenom = get("prenom")
        nom = get("nom")
        nom_usage = get("nom_usage", " ")
        formatted_nom_usage = " " + nom_usage.lower() if nom_usage else ""
        formatted_sigle = ", "+ sigle if sigle else ""
        
        if (prenom is None and nom is None):
            return None
        else:
            return f'{prenom}{formatted_nom_usage} ({nom}{formatted_sigle})'.lower()
    else:
        nom_raison_sociale = get("nom_raison_sociale")
        
        if nom_raison_sociale is None and sigle is None:
            return None
        else:
            formatted_sigle = f' ({sigle})' if sigle else ""
            return f'{nom_raison_sociale}{formatted_sigle}'.lower()

In [ ]:
def parse_unite(cols, row, all_unite_legale, index):
    def get(x, default=None): 
        val = row[cols[x]]
        if not val:
            return default
        return val
    start_time = time.time()

    siren = get('siren')
    unite_legale = get_key(siren, all_unite_legale, {})

    
    unite_legale["nom_complet"] = nom_complet(cols, row)
    col_list = ["date_creation_unite_legale", "sigle", "prenom", "tranche_effectif_salarie_unite_legale", "date_mise_a_jour_unite_legale", "categorie_entreprise",
                "etat_administratif_unite_legale", "nom", "nom_usage", "nom_raison_sociale", "nature_juridique_unite_legale", "activite_principale_unite_legale",
                "economie_sociale_solidaire_unite_legale", "identifiant_association_unite_legale"]
    
    for col in col_list:
        unite_legale[col] = get(col)

    activite_principale_unite_legale = get("activite_principale_unite_legale", '')
    code_naf = activite_principale_unite_legale[:2]
    unite_legale["section_activite_principale"] = sections_NAF[code_naf] if code_naf in sections_NAF else None
    unite_legale["is_entrepreneur_individuel"] = True if unite_legale["nature_juridique_unite_legale"] in ['1', '10', '1000'] else False
    
    # unite_legale["concat_nom_adr_siren"] = " ".join(filter(None, [unite_legale["nom_complet"], (" ".join(unite_legale["liste_adresse"]).strip()), siren]))

    all_unite_legale[siren] = unite_legale
    
    if(index % 5000000 == 0):
        stats()
        print(index)

In [ ]:
%%time

with open(unite_file, 'r') as read_obj:
    cols = {}
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for index, row in enumerate(csv_reader):
        if(index==0):
            cols = { val: index for (index, val) in enumerate(row)}
        else:
            parse_unite(cols, row, all_unite_legale, index)

start_time = time.time()
for siren, unite_legale in all_unite_legale.items():
    unite_legale["concat_nom_adr_siren"] = filter(None, unite_legale["nom_complet"] + (" ".join(unite_legale["liste_adresse"]).strip()) + siren) if (unite_legale["nom_complet"] not None) else (" ".join(unite_legale["liste_adresse"]).strip()) + siren 
stats()
start_time = time.time()
for siren, unite_legale in all_unite_legale.items():
    unite_legale["concat_nom_adr_siren"] = " ".join(filter(None, [unite_legale["nom_complet"], (" ".join(unite_legale["liste_adresse"]).strip()), siren]))
stats()

In [ ]:
pprint.pprint(list(all_unite_legale.values())[0:5])

In [ ]:
len(all_unite_legale)

In [ ]:
# test = pd.DataFrame.from_dict(all_unite_legale)

In [ ]:
# test.head(5)